In [8]:
import yfinance as yf
from pandas_datareader import data as dt
import numpy as np
import pandas as pd

In [9]:
# Data 1: import stock market data - S&P and sectors ETF
yf.pdr_override() 

# using different ETF to represents market indices by sector
sectors = {'SPY': 'S&P500', 'XLK': 'Information Technology', 'XLY': 'Consumer Discretionary', 'XLB': 'Materials',
           'XLC': 'Communication Services', 'XLV': 'Health Care', 'XLI': 'Industrials', 'XLP': 'Consumer Staples', 
           'XLF': 'Financial Services', 'XLU': 'Utilities', 'XLRE': 'Real Estate', 'XLE': 'Energy'}
symbols = sectors.keys()
data_source ='google'
start_date = '2019-01-01'
end_date = '2020-12-31'

appended_data = []
for symbol in sectors:
    data = dt.get_data_yahoo(symbol, start_date, end_date)
    data['symbol'] = symbol
    data['sector'] = sectors[symbol]
    appended_data.append(data)
SNP = pd.concat(appended_data)
SNP['Date'] = SNP.index

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [10]:
# Data 2: Market shares of different sectors
# data source: https://advisor.visualcapitalist.com/sp-500-sectors-and-industries/
# the website has forbidden server access, hence the data is manually pulled and stored in csv
percent = pd.read_csv('./../../data/raw/percent_sector.csv')

# prepare the YoY price change for the sector ETF
# starting time: 2020-01-02
# ending time: end of year 2020-12-30
# drop during COVID: (min-max)/max price during 2020-01-01 - 2020-03-31
snp_soy = SNP[SNP['Date'] == '2020-01-02']
snp_eoy = SNP[SNP['Date'] == '2020-12-30']
percent = percent.merge(snp_soy[['Close','sector']], how = 'left', left_on = 'Sector', right_on = 'sector')
percent = percent.rename(columns={"Close": "price_soy"})
percent = percent.merge(snp_eoy[['Close','sector']], how = 'left', left_on = 'Sector', right_on = 'sector')
percent = percent.rename(columns={"Close": "price_eoy"})
percent['change_yoy'] = (percent['price_eoy']-percent['price_soy'])/percent['price_soy']

snp_covid = SNP.loc[(SNP['Date'] <= '2020-03-31') & (SNP['Date'] >= '2020-01-01')]
snp_covid_max = snp_covid.groupby(['sector'], as_index=False).max()
snp_covid_min = snp_covid.groupby(['sector'], as_index=False).min()
percent = percent.merge(snp_covid_max[['Close','sector']], how = 'left', left_on = 'Sector', right_on = 'sector')
percent = percent.rename(columns={"Close": "price_max"})
percent = percent.merge(snp_covid_min[['Close','sector']], how = 'left', left_on = 'Sector', right_on = 'sector')
percent = percent.rename(columns={"Close": "price_min"})
percent['drop_covid'] = (percent['price_min']-percent['price_max'])/percent['price_max']

percent = percent[['Sector', 'Percent', 'price_soy', 'price_eoy', 'change_yoy', 'price_max', 'price_min', 'drop_covid']]

In [11]:
# Data 3: AstraZeneca (AZN), Pfizer (PFE), Johnson & Johnson (JNJ), Moderna (MRNA), Novavax (NVAX) Stock price
yf.pdr_override() 

companies = {'AZN': 'AstraZeneca', 'PFE': 'Pfizer', 'JNJ': 'Johnson & Johnson', 'MRNA': 'Moderna', 'NVAX': 'Novavax'}
vaccine_symbols = companies.keys()
data_source='google'
start_date = '2019-01-01'
end_date = '2020-12-31'

vaccine = []
for symbol in companies:
    data = dt.get_data_yahoo(symbol, start_date, end_date)
    data['symbol'] = symbol
    data['sector'] = companies[symbol]
    vaccine.append(data)
vaccine_stock = pd.concat(vaccine)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [12]:
# Data 4. Vaccine order data
# data source: https://launchandscalefaster.org/covid-19#Interactive%20tables%20and%20charts%20-%20COVID-19%20Vaccine%20Advance%20Market%20Commitments
# by looking at the html, found the source public Tableau dashboard and download the data from there
# Tableau view: https://public.tableau.com/views/COVID-19VaccinePurchase_16099487574570/TotalConfirmedDosesbyCountryandVaccineCandidate?:embed=y&:showVizHome=no&:host_url=https%3A%2F%2Fpublic.tableau.com%2F&:embed_code_version=3&:tabs=yes&:toolbar=yes&:animate_transition=yes&:display_static_image=no&:display_spinner=no&:display_overlay=yes&:display_count=yes&:language=en&publish=yes&:loadOrderID=1

vaccine_purchase_all = pd.read_csv('./../../data/raw/purchasing_deals.csv')
vaccine_purchase = vaccine_purchase_all[['Company','Partners','Purchaser Entity / Country', "Purchaser's country Economic  Status",
                                         'Number of Doses Procured']]

conditions = [
    (vaccine_purchase['Partners'] == 'AstraZeneca'),
    (vaccine_purchase['Company'] == 'Pfizer'),
    (vaccine_purchase['Company'] == 'Moderna'),
    (vaccine_purchase['Company'] == 'Janssen (J&J)'),
    (vaccine_purchase['Company'] == 'Novavax')
    ]

values_stock = ['AZN', 'PFE', 'MARNA', 'JNJ', 'NVAX']
values_brand = ['AstraZeneca', 'Pfizer', 'Moderna', 'Johnson & Johnson', 'Novavax']

vaccine_purchase['stock'] = np.select(conditions, values_stock, default='Other')
vaccine_purchase['brand'] = np.select(conditions, values_brand, default='Other')

vaccine_purchase = vaccine_purchase.rename(columns={'Purchaser Entity / Country': "country", 
                                                    "Purchaser's country Economic  Status": "economic status",
                                                    "Number of Doses Procured": "doses"})

vaccine_country = vaccine_purchase.groupby('country', as_index=False).sum()
vaccine_brand = vaccine_purchase.groupby('brand', as_index=False).sum()

C:\Users\tiger\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tiger\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
SNP.to_csv('./../../data/clean/SNP.csv')
percent.to_csv('./../../data/clean/percent_sector.csv')
vaccine_stock.to_csv('./../../data/clean/vaccine_stock.csv')
vaccine_brand.to_csv('./../../data/clean/vaccine_brand.csv')
vaccine_country.to_csv('./../../data/clean/vaccine_country.csv')

In [14]:
# in app code, there should be following import code:
percent = pd.read_csv('./../../data/clean/percent_sector.csv')
SNP = pd.read_csv('./../../data/clean/SNP.csv')
vaccine_stock = pd.read_csv('./../../data/clean/vaccine_stock.csv')
vaccine_brand = pd.read_csv('./../../data/clean/vaccine_brand.csv')
vaccine_country = pd.read_csv('./../../data/clean/vaccine_country.csv')